# Preparation of test atmosphere profiles
The test atmosphere profiles are received in a text file, which will be converted to the fortran format that is compatible with `lblnew-bestfit` and `clirad-lw`.

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [285]:
from pathlib import *
import pickle, re, pprint, calendar
import numpy as np, pandas as pd, xarray as xr
from IPython import display

from bokeh.palettes import all_palettes
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import Range1d

output_notebook()

Loading BokehJS ...

In [3]:
PATH = Path('/chia_cluster/home/jackyu/radiation/crd/atmosphere_profiles')

## Txt to Dataset

In [5]:
atmtxt = open(PATH/'era5_clearsky_value.txt', mode='r', encoding='utf-8').read()

In [26]:
atmtxts = atmtxt.split("=\n=")

In [85]:
_re_name = r'Grid Name = (.*)'
_re_date = r'Grid Date = (.*)'
_re_coords = r'Grid Coordinate Lon = (.*) \| Lat = (.*)'
_re_cloud = r'Total Cloud = (.*)'
_re_tsfc = r'Skin Temp.  = (.*)'
_re_pt = r'Vert.(.*)hPa Temp. = (.*)'
_re_pq = r'Vert.(.*)hPa Q.    = (.*)'

def get_wlayer(s):
    m = re.compile(_re_pq)
    player, wlayer = zip(*m.findall(s))
    player = np.array(player, dtype=float)
    wlayer = np.array(wlayer, dtype=float)
    return xr.DataArray(wlayer, coords=[('player', player),])

def get_tlayer(s):
    m = re.compile(_re_pt)
    player, tlayer = zip(*m.findall(s))
    player = np.array(player, dtype=float)
    tlayer = np.array(tlayer, dtype=float)
    return xr.DataArray(tlayer, coords=[('player', player)])

def get_atmpro(s):
    ds = xr.Dataset()
    
    ds.attrs['name'] = re.compile(_re_name).findall(s)[0]
    
    ts = re.compile(_re_date).findall(s)[0]
    ds.attrs['datetime'] = pd.Timestamp(ts[:4] + '-' + ts[4:6] + '-' + ts[6:8] + ' ' + ts[8:])
    
    lon, lat = re.compile(_re_coords).findall(s)[0]
    lon, lat = float(lon), float(lat)
    ds.attrs['longitude'], ds.attrs['latitude']  = lon, lat
    
    ds.attrs['cloud'] = float(re.compile(_re_cloud).findall(s)[0])
    ds.attrs['tsfc'] = float(re.compile(_re_tsfc).findall(s)[0])
    ds['tlayer'] = get_tlayer(s)
    ds['wlayer'] = get_wlayer(s)
    return ds

In [96]:
atmpros = [get_atmpro(s) for s in  atmtxts]

In [99]:
with open(PATH/'era37.pkl', mode='wb') as f:
    pickle.dump(atmpros, f)

In [105]:
atmpros = pickle.load(open(PATH/'era37.pkl', mode='rb'))

## Plots
Let's take a look at some of these profiles.

In [173]:
all_palettes['Paired'].keys();

In [168]:
colors = all_palettes['Paired'][len(atmpros)]

In [174]:
def atmpro_to_legend(atmpro):
    name = '.'.join(w[0].upper() for w in atmpro.attrs['name'].split())
    lon, lat = atmpro.attrs['longitude'], atmpro.attrs['latitude']
    ts = atmpro.attrs['datetime']
    tsfc = atmpro.attrs['tsfc']
    return f"{name} ({lon:.1f},{lat:.1f}) {ts.month} {tsfc:.0f}K"

def get_pfunc(p, atmpro):
    name = '.'.join(w[0].upper() for w in atmpro.attrs['name'].split())
    if name.startswith('A.S'): return p.cross
    if name.startswith('A.N'): return p.circle
    if name.startswith('W.P'): return p.triangle
    if name.startswith('C.P'): return p.asterisk
    
def plt_vlayer(vname):
    p = figure(x_axis_type='log', y_axis_type='log', title=vname.upper())
    for atmpro, c in zip(atmpros[:], colors[:]):
        player, vlayer = atmpro.player.values, atmpro[vname].values
    
        legend = atmpro_to_legend(atmpro)
        pfunc = get_pfunc(p, atmpro)
        
        pfunc(vlayer, player, color=c, size=np.random.randint(10, 20), alpha=.7, legend=legend)
    
        p.yaxis.axis_label = 'Pressure (hPa)'
        p.y_range = Range1d(player.max(), player.min())
    
        p.xaxis.axis_label = f'{vname}'
    
        p.legend.location = 'center_right'
    return p

In [175]:
show(plt_vlayer('tlayer'))

In [176]:
show(plt_vlayer('wlayer'))

## plevel 
Get `plevel` by linearly interpolating `log(player)`.

In [321]:
df = pd.DataFrame(atmpros[0].player.values, columns=['player'])
df['layer'] = np.arange(df.shape[0]) + 1
df = df[['layer', 'player']]
df['logp'] = np.log(df['player'])
df = df[['layer', 'logp', 'player']]
df

,layer,logp,player
0,1,0.000000,1.0
1,2,0.693147,2.0
2,3,1.098612,3.0
3,4,1.609438,5.0
4,5,1.945910,7.0
5,6,2.302585,10.0
6,7,2.995732,20.0
7,8,3.401197,30.0
8,9,3.912023,50.0
9,10,4.248495,70.0


In [320]:
r = df['logp'].rolling(2)
dfl = pd.DataFrame(r.mean())
dfl['plevel'] = np.exp(dfl.logp)
dfl['plevel'].iloc[0] = 0
dfl = dfl.append({'plevel':1013}, ignore_index=True)
dfl['level'] = dfl.index.values + 1
dfl = dfl[['level', 'logp', 'plevel']]
dfl

,level,logp,plevel
0,1,NaN,0.000000
1,2,0.346574,1.414214
2,3,0.895880,2.449490
3,4,1.354025,3.872983
4,5,1.777674,5.916080
5,6,2.124248,8.366600
6,7,2.649159,14.142136
7,8,3.198465,24.494897
8,9,3.656610,38.729833
9,10,4.080259,59.160798


In [240]:
for ds in atmpros: ds['plevel'] = ('plevel', dfl['plevel'].values)

In [324]:
ds = atmpros[0]
ps = np.stack([ds.player.values, ds.plevel.values[1:]], axis=1).flatten()
ps = np.array([0] + list(ps))
p = figure(plot_width=500, plot_height=300, title="player and plevel")
p.circle(x=range(len(ps)), y=ps)
p.yaxis.axis_label = 'pressure (hPa)'
p.xaxis.axis_label = 'index'
show(p)

In [317]:
with pd.ExcelWriter('player_plevel.xlsx') as writer:
    df.to_excel(writer, sheet_name='player')
    dfl.to_excel(writer, sheet_name='plevel')

## Add `olayer`
So, set these to 0, for now.

In [260]:
for o in atmpros:
    o['olayer'] = ('player', np.zeros(o.player.shape))

In [266]:
atmpros[2].attrs, atmpros[3].attrs

(OrderedDict([('name', 'ARM NORTH SLOPE OF ALASKA'),
              ('datetime', Timestamp('2018-06-11 07:00:00')),
              ('longitude', 156.609),
              ('latitude', 71.323),
              ('cloud', 0.0),
              ('tsfc', 274.725)]),
 OrderedDict([('name', 'ARM NORTH SLOPE OF ALASKA'),
              ('datetime', Timestamp('2014-12-11 05:00:00')),
              ('longitude', 156.609),
              ('latitude', 71.323),
              ('cloud', 0.0),
              ('tsfc', 248.18)]))

In [267]:
with open(PATH/'era37.pkl', 'wb') as f: pickle.dump(atmpros, f)

In [269]:
atmpros = pickle.load(open(PATH/'era37.pkl', 'rb'))

## Names to be used for the profiles
So far each test profile is identified by its location, time and surface temperature. Let's make up short names based on these for identifying each profile, like how 'mls' stands for mid-latitude summer.

In [307]:
def get_atmpro_name(ds):
    sname = ''.join(w[0].upper() for w in ds.attrs['name'].split())
    month = calendar.month_abbr[ds.attrs['datetime'].month]
    return '_'.join([sname, month, f"{len(ds.player)}"])

Here are the shorthand name and surface temperature for each test profile.

In [309]:
for o in atmpros: print(get_atmpro_name(o), o.attrs['tsfc'])

ASGP_Jun_37 302.724
ASGP_Dec_37 281.162
ANSOA_Jun_37 274.725
ANSOA_Dec_37 248.18
WPTWP_Jun_37 302.62
WPTWP_Dec_37 301.831
CPTEP_Jun_37 299.699
CPTEP_Dec_37 295.963


# fin

In [325]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')